In [5]:
import subprocess as sp
import numpy as np
import nibabel as nib
from util_func import *

from scipy.ndimage import binary_erosion

sub_sm = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06',
          'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11',
          'sub-12', 'sub-13', 'sub-14', 'sub-15', 
          'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21',
          'sub-22', 'sub-23', 'sub-24', 'sub-25',
          'sub-26', 'sub-27', 'sub-28']

## Excluding subjects with excessive motion

In [9]:
sub_sm_clean = []
for subj in sub_sm:
    for task in ['movie', 'qna']:
        for run in range(1,4):
            try:
                df = load_confounds(subj, task=task, run=run)
                df = extract_cols(df)
                fd = np.array(df['framewise_displacement'].values.tolist())

                framewise_displacement = fd
                percent_motion_outliers = np.sum(framewise_displacement>0.5)/fd.shape[0]
                print(subj,task,run,': ',percent_motion_outliers)
                if percent_motion_outliers > 0.05:
                    sub_sm_clean.append([subj,task,run, percent_motion_outliers])
                    # print(subj,task,run,': ',percent_motion_outliers)
            except:
                pass
          

sub-01 movie 1 :  0.012048192771084338
sub-01 movie 2 :  0.009394572025052192
sub-01 movie 3 :  0.007042253521126761
sub-01 qna 1 :  0.0
sub-01 qna 2 :  0.0010526315789473684
sub-01 qna 3 :  0.0014792899408284023
sub-02 movie 1 :  0.0
sub-02 movie 2 :  0.0
sub-02 movie 3 :  0.0
sub-02 qna 1 :  0.0
sub-02 qna 2 :  0.0
sub-02 qna 3 :  0.0
sub-03 movie 1 :  0.0
sub-03 movie 2 :  0.0
sub-03 movie 3 :  0.0
sub-03 qna 1 :  0.0
sub-03 qna 2 :  0.0
sub-03 qna 3 :  0.006042296072507553
sub-04 movie 1 :  0.0
sub-04 movie 2 :  0.0
sub-04 movie 3 :  0.0
sub-04 qna 1 :  0.0
sub-04 qna 2 :  0.0
sub-04 qna 3 :  0.0
sub-05 movie 1 :  0.0
sub-05 movie 2 :  0.0020876826722338203
sub-05 movie 3 :  0.0
sub-05 qna 1 :  0.0
sub-05 qna 2 :  0.0
sub-05 qna 3 :  0.0
sub-07 movie 1 :  0.0
sub-07 movie 2 :  0.0020876826722338203
sub-07 movie 3 :  0.0
sub-07 qna 1 :  0.0
sub-07 qna 2 :  0.002663115845539281
sub-07 qna 3 :  0.0011350737797956867
sub-08 movie 1 :  0.0
sub-08 movie 2 :  0.005219206680584551
sub-08 m

## Denosing confounds

In [25]:
sub_sm_exclude = []
folder_name = f'/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie1'

for sub in sub_sm:
    for task in ['movie','qna']:
        for run in range(1,4):
            print('+++++++++', sub, task, run, '+++++++++')
            # path = f'{folder_name}/derivatives/{sub}/func/{sub}_task-{task}_run-{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
            path = f'{folder_name}/derivatives/{sub}/func/{sub}_task-{task}_run-{run:02d}_space-T1w_desc-preproc_bold.nii.gz'

            mni_mask = nib.load(f'{folder_name}/preprocessed_surf/{sub}_cortical_mask.nii.gz').get_fdata()
            mni_image = nib.load(f'{folder_name}/preprocessed_surf/{sub}_cortical_mask.nii.gz')

            fmri_data = nib.load(path).get_fdata()
            print('original: ', fmri_data.shape)
            fmri_data = fmri_data[mni_mask==1,:].T
            print('masked: ', fmri_data.shape)

            fmri_compounds = load_confounds(sub, task, run)
            fmri_clean = clean_data(fmri_data, fmri_compounds)
            ffmri_clean = fmri_clean.astype(np.float32)

            new_image = np.zeros(mni_mask.shape+(fmri_data.shape[0],), dtype=np.float32)
            new_image[mni_mask==1,:] = fmri_clean.T

            nib.save(nib.Nifti1Image(new_image, mni_image.affine), f'{folder_name}/preprocessed_surf/{sub}_{task}_{run}.nii.gz')


+++++++++ sub-01 movie 1 +++++++++
original:  (48, 60, 48, 996)
masked:  (996, 26733)
+++++++++ sub-01 movie 2 +++++++++
original:  (48, 60, 48, 958)
masked:  (958, 26733)
+++++++++ sub-01 movie 3 +++++++++
original:  (48, 60, 48, 710)
masked:  (710, 26733)
+++++++++ sub-01 qna 1 +++++++++
original:  (48, 60, 48, 950)
masked:  (950, 26733)
+++++++++ sub-01 qna 2 +++++++++
original:  (48, 60, 48, 950)
masked:  (950, 26733)
+++++++++ sub-01 qna 3 +++++++++
original:  (48, 60, 48, 1352)
masked:  (1352, 26733)
+++++++++ sub-02 movie 1 +++++++++
original:  (54, 58, 51, 996)
masked:  (996, 28015)
+++++++++ sub-02 movie 2 +++++++++
original:  (54, 58, 51, 958)
masked:  (958, 28015)
+++++++++ sub-02 movie 3 +++++++++
original:  (54, 58, 51, 710)
masked:  (710, 28015)
+++++++++ sub-02 qna 1 +++++++++
original:  (54, 58, 51, 730)
masked:  (730, 28015)
+++++++++ sub-02 qna 2 +++++++++
original:  (54, 58, 51, 635)
masked:  (635, 28015)
+++++++++ sub-02 qna 3 +++++++++
original:  (54, 58, 51, 758)


## Additional preprocessing

In [ ]:
# Additional preprocessing step

folder_name = f'/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie1/preprocessed_surf'
FWHM = 6
for sub in sub_sm:
    for task in ['movie']:
        for run in range(1,4):
            print(sub,task,run)
            mask_fname = f'{folder_name}/{sub}_cortical_mask.nii.gz'
            input_fname = f'{folder_name}/{sub}_{task}_{run}.nii.gz'
            mean_fname = f'{folder_name}/{sub}_{task}_{run}_mean.nii.gz'
            SC_fname = f'{folder_name}/{sub}_{task}_{run}_sc.nii.gz'
            SM_SC_fname = f'{folder_name}/{sub}_{task}_{run}_sc_sm.nii.gz'

            sp.call(f'/home/dasom/abin/3dTstat -prefix {mean_fname} {input_fname}', shell=True) # Mean calculation
            sp.call(f"/home/dasom/abin/3dcalc -a {input_fname} -b {mean_fname} -c {mask_fname} -expr 'c * min(200, a/b*100)*step(a)*step(b)' -prefix {SC_fname}", shell=True) # Scaling
            sp.call(f'/home/dasom/abin/3dmerge -quiet -1blur_fwhm {FWHM} -doall -prefix {SM_SC_fname} {SC_fname}',  shell=True) # Spatial Smoothing